In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit, cross_val_score

from sklearn.linear_model import LinearRegression, ElasticNet, BayesianRidge, SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, AdaBoostRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor

from sklearn.svm import LinearSVR

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import datetime
from tqdm import tqdm_notebook

# Load

In [37]:
X = np.load('data/X.npy')
y = np.load('data/y.npy')

# Finding best model
Et tu, Bruteforce?

In [3]:
def calculate_cv(X, y):
    cv = TimeSeriesSplit(n_splits=4)
    results = {
        'lr': [],
        'el_net': [],
        'bayes_ridge': [],
        'sgd': [],
        'gbr': [],
        'rf': [],
        'ext': [],
        'bag_reg': [],
        'abr': [],
        'kn': [],
        'dtr': [],
        'etr': [],
        'svm': [],
        'xgb': [],
        'lgb': [],
        'cat': [],
    }
    
    lr = LinearRegression()
    el_net = ElasticNet()
    bayes_ridge = BayesianRidge()
    sgd = SGDRegressor()
    
    gbr = GradientBoostingRegressor() 
    rf = RandomForestRegressor()
    ext = ExtraTreeRegressor()
    bag_reg = BaggingRegressor()
    abr = AdaBoostRegressor()
    
    kn = KNeighborsRegressor(n_neighbors = 3, weights='uniform', p = 1)
    
    dtr = DecisionTreeRegressor()
    etr = ExtraTreeRegressor()
    
    svm = LinearSVR()
    
    xgb = XGBRegressor()
    lgb = LGBMRegressor()
    cat = CatBoostRegressor()
    
    results['lr'].append(cross_val_score(lr, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['el_net'].append(cross_val_score(el_net, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['bayes_ridge'].append(cross_val_score(bayes_ridge, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['sgd'].append(cross_val_score(sgd, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['gbr'].append(cross_val_score(gbr, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['rf'].append(cross_val_score(rf, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['ext'].append(cross_val_score(ext, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['bag_reg'].append(cross_val_score(bag_reg, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['abr'].append(cross_val_score(abr, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['kn'].append(cross_val_score(kn, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['dtr'].append(cross_val_score(dtr, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['etr'].append(cross_val_score(etr, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['svm'].append(cross_val_score(svm, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['xgb'].append(cross_val_score(xgb, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())
    results['lgb'].append(cross_val_score(lgb, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1, fit_params = {}).mean())
    results['cat'].append(cross_val_score(cat, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1).mean())    
    
    for key in list(results.keys()):
        print (f'Mean absolute error: {abs(results[key][0])}, algorithm: {key}')

In [4]:
calculate_cv(X,y)

Mean absolute error: 48.2986788097265, algorithm: lr
Mean absolute error: 36.80135163757205, algorithm: el_net
Mean absolute error: 32.450379000236154, algorithm: bayes_ridge
Mean absolute error: 4088007294844225.5, algorithm: sgd
Mean absolute error: 43.74809939966454, algorithm: gbr
Mean absolute error: 41.119642857142864, algorithm: rf
Mean absolute error: 27.29017857142857, algorithm: ext
Mean absolute error: 44.824553571428574, algorithm: bag_reg
Mean absolute error: 49.766815358044965, algorithm: abr
Mean absolute error: 15.930059523809522, algorithm: kn
Mean absolute error: 46.98214285714286, algorithm: dtr
Mean absolute error: 44.89732142857142, algorithm: etr
Mean absolute error: 37.60330003163385, algorithm: svm
Mean absolute error: 44.5396576160565, algorithm: xgb
Mean absolute error: 28.590413747547903, algorithm: lgb
Mean absolute error: 27.593352975861876, algorithm: cat


## Autoregressions

In [5]:
from sklearn.metrics import mean_absolute_error

from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA, ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [6]:
data = pd.read_csv('data/train_target.csv', index_col= 0)

In [7]:
def check_auto_reg(data):
    cv = TimeSeriesSplit(n_splits=4)
    
    score = []
    for train_ind, test_ind in cv.split(data):
        model = AR(data['target'][train_ind])
        model_fit = model.fit()
        yhat = model_fit.predict(start = str(data.index[test_ind[0]]), end = str(data.index[test_ind[-1]]))
        score.append(mean_absolute_error(data['target'][test_ind], yhat))
    print ('Autoregression: ', np.mean(score))
    
    score = []
    for train_ind, test_ind in cv.split(data):
        model = ARMA(data['target'][train_ind], order=(0, 1))
        model_fit = model.fit()
        yhat = model_fit.predict(start = str(data.index[test_ind[0]]), end = str(data.index[test_ind[-1]]))
        score.append(mean_absolute_error(data['target'][test_ind], yhat))
    print ('Moving Average: ', np.mean(score))
    
    score = []
    for train_ind, test_ind in cv.split(data):
        model = ARIMA(data['target'][train_ind], order=(1, 1, 1))
        model_fit = model.fit(disp=False)
        yhat = model_fit.predict(start = str(data.index[test_ind[0]]), end = str(data.index[test_ind[-1]]))
        score.append(mean_absolute_error(data['target'][test_ind], yhat))
    print ('Autoregressive Integrated Moving Average: ', np.mean(score))

In [8]:
check_auto_reg(data)

/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MO

Autoregression:  46.56476301663901
Moving Average:  43.19917643445923


/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MO

Autoregressive Integrated Moving Average:  25.15649791511033


/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)
/home/denis/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  % freq, ValueWarning)


## A bit feature-engineering

In [9]:
from sklearn.cluster import KMeans

In [38]:
X_clus = np.concatenate((KMeans(3).fit_transform(X,y), X), axis=1)

In [11]:
calculate_cv(X_clus,y)

Mean absolute error: 45.08147507897011, algorithm: lr
Mean absolute error: 32.373093070777244, algorithm: el_net
Mean absolute error: 28.02465636019223, algorithm: bayes_ridge
Mean absolute error: 6254070102932415.0, algorithm: sgd
Mean absolute error: 32.227814492090154, algorithm: gbr
Mean absolute error: 37.44642857142857, algorithm: rf
Mean absolute error: 46.05357142857143, algorithm: ext
Mean absolute error: 39.33928571428571, algorithm: bag_reg
Mean absolute error: 38.86564891009197, algorithm: abr
Mean absolute error: 17.08482142857143, algorithm: kn
Mean absolute error: 41.34375, algorithm: dtr
Mean absolute error: 21.522321428571427, algorithm: etr
Mean absolute error: 43.44484351310867, algorithm: svm
Mean absolute error: 32.8727540006595, algorithm: xgb
Mean absolute error: 20.669529125413288, algorithm: lgb
Mean absolute error: 19.518386711670676, algorithm: cat


In [39]:
X_clus_num = np.concatenate((KMeans().fit_transform(pd.read_csv('data/train_num_cols.csv').values, y), X), axis=1) 

In [13]:
calculate_cv(X_clus_num, y)

Mean absolute error: 47.122344856870576, algorithm: lr
Mean absolute error: 32.98369070247763, algorithm: el_net
Mean absolute error: 30.705852229412223, algorithm: bayes_ridge
Mean absolute error: 5104723973062055.0, algorithm: sgd
Mean absolute error: 41.93763662981808, algorithm: gbr
Mean absolute error: 40.98571428571428, algorithm: rf
Mean absolute error: 48.83482142857143, algorithm: ext
Mean absolute error: 44.575892857142854, algorithm: bag_reg
Mean absolute error: 48.76214003410852, algorithm: abr
Mean absolute error: 16.941964285714285, algorithm: kn
Mean absolute error: 40.558035714285715, algorithm: dtr
Mean absolute error: 44.848214285714285, algorithm: etr
Mean absolute error: 30.673275612551794, algorithm: svm
Mean absolute error: 42.45413471119745, algorithm: xgb
Mean absolute error: 34.77319411196585, algorithm: lgb
Mean absolute error: 27.84343806055621, algorithm: cat


## Bonus (just for fun)

In [29]:
import torch

In [68]:
device = torch.device('cuda')

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
D_in, D_out = 15, 1

X_tensor = torch.tensor(X, device=device)
y_tensor = torch.tensor(y, device=device)

class Net(nn.Module):

    def __init__(self, input_dim, hidden_dims):
        super(Net, self).__init__()
        layers = []
        
        for i in range(len(hidden_dims)):
            layers.append(nn.Linear(input_dim, hidden_dims[i]))
            layers.append(nn.ReLU())
            # layers.append(nn.LSTM(input_dim, hidden_dims[i], i))
            input_dim = hidden_dims[i]
        
        
        self.net = nn.Sequential(*layers)
        self.out = nn.Linear(input_dim, 1)
        
        
    def forward(self, x):
        x = self.out(self.net(x))
        
        return x

model = Net(D_in, [4, 8, 16]).to(device)

model = model.double()

loss_fn = torch.nn.L1Loss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

best_loss = np.inf
losses = []
for t in range(200000):
    y_pred = model(x)

    loss = loss_fn(y_pred, y_tensor.double().view(282, 1))
    if (t+1)%10000 == 0: 
        print(t+1, loss.item())
    
    if best_loss > loss:
        best_loss = loss
        torch.save({'state_dict': model.state_dict()}, 'checkpoint.pth')
        
    losses.append(np.float(loss))
    model.zero_grad()
    loss.backward()
    optimizer.step()
    
print (best_loss)

10000 8470.28891199592
20000 8470.128911995951
30000 8470.108808014062
40000 8470.528911996014
50000 8470.368911996045
60000 8470.208911996078
70000 8470.04891199611
80000 8470.38880801351
90000 8470.44891199617
100000 8470.288911996202
110000 8470.128911996235
120000 8470.108808013067
130000 8470.528911996298
140000 8470.368911996331
150000 8470.20891199636
160000 8470.048911996393
170000 8470.388808012514
180000 8470.448911996456
190000 8470.28891199649
200000 8470.12891199652
tensor(8470.0489, device='cuda:0', dtype=torch.float64, grad_fn=<L1LossBackward>)


Not worked :(

## Hyperparam tuning

In [14]:
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import colorama, hyperopt

import catboost

In [15]:
N_HYPEROPT_PROBES = 1000

HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

colorama.init()

X_test, y_test = X_clus[-30:], y[-30:]
X_train, X_val, y_train, y_val = train_test_split(X_clus[:-30], y[:-30], test_size=0.2, shuffle = False)

D_train = catboost.Pool(X_train, y_train)
D_val = catboost.Pool(X_val, y_val)

In [16]:
def get_catboost_params(space):
    params = dict()
    params['iterations'] = space['iterations']
    params['learning_rate'] = space['learning_rate']
    params['depth'] = int(space['depth'])
    params['l2_leaf_reg'] = space['l2_leaf_reg']
    params['rsm'] = space['rsm']
    params['ctr_leaf_count_limit'] = space['ctr_leaf_count_limit']
    params['bootstrap_type'] = space['bootstrap_type']
    #params['subsample'] = space['subsample']
    return params


obj_call_count = 0
cur_best_loss = np.inf
log_writer = open( 'catboost-hyperopt-log.txt', 'w' )

def objective(space):
    global obj_call_count, cur_best_loss

    obj_call_count += 1

    #print('\nCatBoost objective call #{} cur_best_loss={:7.5f}'.format(obj_call_count,cur_best_loss))

    params = get_catboost_params(space)

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    #print('Params: {}'.format(params_str) )

    model = catboost.CatBoostRegressor(loss_function = 'MAE',
                                       store_all_simple_ctr = True,
                                       early_stopping_rounds=10,
                                       iterations = params['iterations'],
                                       learning_rate = params['learning_rate'],
                                       depth = params['depth'],
                                       l2_leaf_reg = params['l2_leaf_reg'],
                                       rsm = params['rsm'],
                                       ctr_leaf_count_limit = params['ctr_leaf_count_limit'],
                                       bootstrap_type = params['bootstrap_type'],
                                       random_seed = 42,
                                       #subsample = params['subsample'],
                                      )
    
    model.fit(D_train, eval_set=D_val, verbose=False)

    y_pred = model.predict(X_test)

    test_loss = mean_absolute_error(y_test, y_pred)
    
    log_writer.write('MAE={}\n'.format(test_loss))
    log_writer.flush()

    if test_loss<cur_best_loss:
        cur_best_loss = test_loss
        print(colorama.Fore.GREEN + 'NEW BEST LOSS={}'.format(cur_best_loss) + colorama.Fore.RESET)
        print('Params: {}'.format(params_str) )


    return{'loss':test_loss, 'status': STATUS_OK }

In [17]:
space ={
        'iterations': hp.quniform('iterations', 50, 1000, 20),
        'depth': hp.quniform("depth", 4, 12, 1),
        'rsm': hp.uniform ('rsm', 0.5, 1.0),
        'learning_rate': hp.loguniform('learning_rate', -3.0, -0.7),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
        'ctr_leaf_count_limit': hp.quniform('ctr_leaf_count_limit', 1, 20, 1),
        'bootstrap_type': hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']),
        #'subsample': hp.uniform('subsample', .0, 1.0),
       }

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials, verbose = 2)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

NEW BEST LOSS=20.861983127285733
Params: bootstrap_type=Bernoulli ctr_leaf_count_limit=11.0 depth=10.0 iterations=340.0 l2_leaf_reg=5.845430869436142 learning_rate=0.3826332485512357 rsm=0.7962306140036166
NEW BEST LOSS=18.7288824071468
Params: bootstrap_type=Bernoulli ctr_leaf_count_limit=5.0 depth=5.0 iterations=340.0 l2_leaf_reg=2.7185698896238755 learning_rate=0.3230156328989772 rsm=0.6138346001411772
NEW BEST LOSS=17.46315801867186
Params: bootstrap_type=Bernoulli ctr_leaf_count_limit=9.0 depth=4.0 iterations=840.0 l2_leaf_reg=3.6104446734595887 learning_rate=0.2005468311301181 rsm=0.8910682940126395
NEW BEST LOSS=17.400122400473396
Params: bootstrap_type=Bernoulli ctr_leaf_count_limit=13.0 depth=4.0 iterations=980.0 l2_leaf_reg=4.123677219946936 learning_rate=0.18389737399099534 rsm=0.70991086524094
NEW BEST LOSS=16.733745576748053
Params: bootstrap_type=Bernoulli ctr_leaf_count_limit=12.0 depth=4.0 iterations=1000.0 l2_leaf_reg=1.195653898872027 learning_rate=0.17494820817403373

# Predict

In [20]:
X_test = np.load('data/X_test.npy')

km = KMeans(3).fit(X,y) 

X_clus_test = np.concatenate((km.transform(X_test), X_test), axis=1) 

D_train = catboost.Pool(X_clus, y)

model = catboost.CatBoostRegressor(loss_function = 'MAE',
                                   store_all_simple_ctr = True,
                                   early_stopping_rounds=10,
                                   iterations = best['iterations'],
                                   learning_rate = best['learning_rate'],
                                   depth = best['depth'],
                                   l2_leaf_reg = best['l2_leaf_reg'],
                                   rsm = best['rsm'],
                                   ctr_leaf_count_limit = best['ctr_leaf_count_limit'],
                                   bootstrap_type = 'Bernoulli', # yep, its bug
                                   #subsample = params['subsample'],
                                   )

model.fit(D_train)

0:	learn: 34.1663063	total: 821us	remaining: 771ms
1:	learn: 34.0220179	total: 2.44ms	remaining: 1.15s
2:	learn: 33.8784482	total: 3.31ms	remaining: 1.03s
3:	learn: 33.7341604	total: 4.97ms	remaining: 1.16s
4:	learn: 33.5852461	total: 6.89ms	remaining: 1.29s
5:	learn: 33.4406820	total: 9.53ms	remaining: 1.48s
6:	learn: 33.3076455	total: 10.3ms	remaining: 1.37s
7:	learn: 33.1883206	total: 11.7ms	remaining: 1.36s
8:	learn: 33.0661651	total: 14.1ms	remaining: 1.46s
9:	learn: 32.9611975	total: 17.1ms	remaining: 1.59s
10:	learn: 32.8485381	total: 21ms	remaining: 1.78s
11:	learn: 32.7432168	total: 22.9ms	remaining: 1.77s
12:	learn: 32.6386068	total: 24.9ms	remaining: 1.77s
13:	learn: 32.5339584	total: 27ms	remaining: 1.79s
14:	learn: 32.4183953	total: 28.9ms	remaining: 1.78s
15:	learn: 32.3088348	total: 31.2ms	remaining: 1.8s
16:	learn: 32.2056475	total: 32.5ms	remaining: 1.76s
17:	learn: 32.1109149	total: 34.1ms	remaining: 1.75s
18:	learn: 32.0053703	total: 35.4ms	remaining: 1.72s
19:	learn

In [24]:
#predictions:
y_pred = model.predict(X_clus_test)

In [25]:
y_pred.shape

(52,)